## Data Extraction from Different Platform Data Sources



In [40]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# optional 
import warnings
warnings.filterwarnings("ignore")


In [41]:
import pandas as pd

# Base folder path
base_path = r"C:\Users\nisha\OneDrive\Desktop\SPICE_ACADEMY\Marketing_data\data"

# File mapping
files = {
    "meta_mx": "social_meta_mx.csv",
    "meta_non_mx": "social_meta_non_mx.csv",
    "pinterest": "social_pinterest.csv",
    "reddit": "social_reddit.csv"
}

# Load all files into a dictionary of DataFrames
dfs = {}

for name, filename in files.items():
    path = f"{base_path}\\{filename}"
    dfs[name] = pd.read_excel(path)
    print(f"{name}: {dfs[name].shape}")



meta_mx: (81681, 30)
meta_non_mx: (88884, 30)
pinterest: (715, 30)
reddit: (89, 30)


In [42]:
dfs["meta_mx"].head()
dfs["meta_mx"].columns
dfs["meta_mx"].info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81681 entries, 0 to 81680
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Media_Channel__CH             81681 non-null  object        
 1   Type                          81681 non-null  object        
 2   Platform__PL                  79402 non-null  object        
 3   Data_Source                   81681 non-null  object        
 4   Funding_Source__FS            81681 non-null  object        
 5   Sub_Brand__SB                 81681 non-null  object        
 6   Product_Category__PR          81681 non-null  object        
 7   Campaign_Name                 81681 non-null  object        
 8   Line_Item                     0 non-null      float64       
 9   Campaign_Name__CN             81681 non-null  object        
 10  Placement_Description__PD     0 non-null      float64       
 11  Mindset__MD                 

## Cleaning all the dataframes in single loop dropping null values and null coloumns

In [43]:
import re

def extract_campaign_id(text):
    if isinstance(text, str):
        match = re.search(r'ID~([^_]+)', text)
        return match.group(1) if match else None
    return None


In [ ]:
dfs_cleaned = {}

for name, df in dfs.items():
    print(f"\nCleaning {name}...")

    # Drop empty columns
    df = df.dropna(axis=1, how='all')

    # Drop empty rows
    df = df.dropna(how='all')

    # Clean column names
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("__", "_")
        .str.replace("(", "")
        .str.replace(")", "")
    )

    # Extract campaign_id if campaign_name exists
    if "campaign_name" in df.columns:
        df["campaign_id"] = df["campaign_name"].apply(extract_campaign_id)

    # -----------------------------------------
    #Convert numeric-looking columns FIRST
    # -----------------------------------------
    for col in df.columns:
        if col != "date":   # <-- Prevent overwriting date
            df[col] = pd.to_numeric(df[col], errors="ignore")

    # -----------------------------------------
    #Convert nanosecond timestamps to datetime
    # -----------------------------------------
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], unit="ns", errors="coerce")

    # Drop duplicates
    df = df.drop_duplicates()

    # Add platform identifier
    df["platform"] = name

    dfs_cleaned[name] = df

    print(f"{name} cleaned. Shape: {df.shape}")



Cleaning meta_mx...
meta_mx cleaned. Shape: (81681, 29)

Cleaning meta_non_mx...
meta_non_mx cleaned. Shape: (88884, 29)

Cleaning pinterest...
pinterest cleaned. Shape: (715, 28)

Cleaning reddit...
reddit cleaned. Shape: (89, 25)


In [45]:
for name, df in dfs_cleaned.items():
    print(f"\n{name} columns:")
    print(df.columns.tolist())



meta_mx columns:
['media_channel_ch', 'type', 'platform_pl', 'data_source', 'funding_source_fs', 'sub_brand_sb', 'product_category_pr', 'campaign_name', 'campaign_name_cn', 'mindset_md', 'date', 'quarter', 'month', 'cw_iso', 'year', 'division_bs', 'business_activity', 'kpi_pk', 'costs', 'cost_usd', 'impressions', 'clicks', 'total_conversions', 'total_conversions_revenue', 'total_conversion_revenue_usd', 'mobile_app_installs', 'objective_ob', 'campaign_id', 'platform']

meta_non_mx columns:
['media_channel_ch', 'type', 'platform_pl', 'data_source', 'funding_source_fs', 'sub_brand_sb', 'product_category_pr', 'campaign_name', 'campaign_name_cn', 'mindset_md', 'date', 'quarter', 'month', 'cw_iso', 'year', 'division_bs', 'business_activity', 'kpi_pk', 'costs', 'cost_usd', 'impressions', 'clicks', 'total_conversions', 'total_conversions_revenue', 'total_conversion_revenue_usd', 'mobile_app_installs', 'objective_ob', 'campaign_id', 'platform']

pinterest columns:
['media_channel_ch', 'type',

## cleaning done coloumn names has been uniformed case lowered and platform added as a extra column now creating engine to push it to datawarehouse and extracted campaign ID from each data source

In [46]:
POSTGRES_USER = "nishantsingh"
POSTGRES_PASS = "MViOfwCs2ZrSVoV3"
POSTGRES_HOST = "data-analytics-course-2.c8g8r1deus2v.eu-central-1.rds.amazonaws.com"
POSTGRES_PORT = "5432"
POSTGRES_DB = "jalapeno_joins"
POSTGRES_SCHEMA = "s_nishantsingh"


In [47]:
engine = create_engine(
    f"postgresql://{POSTGRES_USER}:{POSTGRES_PASS}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
)


In [48]:
tables_to_drop = [
    "meta_mx",
    "meta_non_mx",
    "pinterest",
    "reddit"
]

with engine.connect() as conn:
    for table in tables_to_drop:
        sql = f'DROP TABLE IF EXISTS {POSTGRES_SCHEMA}.{table} CASCADE'
        print("Dropping:", sql)
        conn.execute(text(sql))
    conn.commit()


Dropping: DROP TABLE IF EXISTS s_nishantsingh.meta_mx CASCADE
Dropping: DROP TABLE IF EXISTS s_nishantsingh.meta_non_mx CASCADE
Dropping: DROP TABLE IF EXISTS s_nishantsingh.pinterest CASCADE
Dropping: DROP TABLE IF EXISTS s_nishantsingh.reddit CASCADE


In [49]:
for name, df in dfs_cleaned.items():
    print(f"Loading {name} with shape {df.shape}")
    df.to_sql(
        name,
        engine,
        schema=POSTGRES_SCHEMA,
        if_exists="replace",
        index=False
    )
    print(f"Loaded {name}")



Loading meta_mx with shape (81681, 29)
Loaded meta_mx
Loading meta_non_mx with shape (88884, 29)
Loaded meta_non_mx
Loading pinterest with shape (715, 28)
Loaded pinterest
Loading reddit with shape (89, 25)
Loaded reddit


In [1]:
dfs_cleaned["meta_mx"].column


NameError: name 'dfs_cleaned' is not defined